In [1]:
import pandas as pd
import utils
import numpy as np
import warnings
from tqdm import tqdm
from copairs.map import average_precision, mean_average_precision
import logging

logging.basicConfig(format="%(levelname)s:%(asctime)s:%(name)s:%(message)s")
logging.getLogger("copairs").setLevel(logging.INFO)

warnings.simplefilter(action="ignore", category=FutureWarning)

c:\Users\ssivagur\Anaconda3\envs\copairs\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#operations = "wellpos_cc_var_mad_outlier_featselect_sphering_harmony"
batch_size = 20000
null_size = 20000
fdr = 0.1

### Prepare the data

#### Read the CRISPR parquet file

In [3]:
#Reading the output from the ORF pipeline
raw_CRISPR_df = pd.read_parquet('C:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\\CRISPRPipelineValidation\\profiles_wellpos_cc_var_mad_outlier_featselect_sphering_harmony_PCA_corrected.parquet')

<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\ssivagur\AppData\Local\Temp\ipykernel_27540\985476001.py:2: SyntaxWarning: invalid escape sequence '\C'
  raw_CRISPR_df = pd.read_parquet('C:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\\CRISPRPipelineValidation\\profiles_wellpos_cc_var_mad_outlier_featselect_sphering_harmony_PCA_corrected.parquet')


In [4]:
raw_CRISPR_df.shape

(946431, 259)

In [5]:
# Filtering only the ORF from the raw output 
orf_df = raw_CRISPR_df[raw_CRISPR_df['Metadata_PlateType'] == 'ORF']

In [6]:
orf_df.shape

(82483, 259)

In [7]:
#dropping the duplicate metadata columns
orf_df_cols = [c for c in orf_df.columns if c.startswith("Metadata_")]

In [8]:
orf_df_cols

['Metadata_Source',
 'Metadata_Plate',
 'Metadata_Well',
 'Metadata_JCP2022',
 'Metadata_broad_sample',
 'Metadata_Name',
 'Metadata_Vector',
 'Metadata_Transcript',
 'Metadata_Symbol_x',
 'Metadata_NCBI_Gene_ID_x',
 'Metadata_Taxon_ID',
 'Metadata_Gene_Description',
 'Metadata_Prot_Match',
 'Metadata_Insert_Length',
 'Metadata_pert_type',
 'Metadata_NCBI_Gene_ID_y',
 'Metadata_Symbol_y',
 'Metadata_Batch',
 'Metadata_PlateType',
 'Metadata_Row',
 'Metadata_Column',
 'Metadata_Microscope',
 'Metadata_Symbol',
 'Metadata_Approved_symbol',
 'Metadata_Locus',
 'Metadata_Chromosome',
 'Metadata_arm']

In [9]:
# there are duplicate columns of NCBI_Gene_ID and the Symbol and the columns with the NaN are dropped
# orf_df['Metadata_NCBI_Gene_ID_y'] has NaN and it is dropped
orf_df['Metadata_NCBI_Gene_ID_y']

334537   NaN
334538   NaN
334539   NaN
334540   NaN
334541   NaN
          ..
417015   NaN
417016   NaN
417017   NaN
417018   NaN
417019   NaN
Name: Metadata_NCBI_Gene_ID_y, Length: 82483, dtype: float64

In [10]:
orf_df['Metadata_NCBI_Gene_ID_y'].isnull().all()

True

In [33]:
orf_df = orf_df.drop(['Metadata_NCBI_Gene_ID_y'], axis=1)

In [11]:
# orf_df['Metadata_Symbol_y'] has NaN and it is dropped
orf_df['Metadata_Symbol_y'].isnull().all()

True

In [12]:
# orf_df['Metadata_Symbol'] has NaN and it is dropped
orf_df['Metadata_Symbol'].isnull().all()

True

In [50]:
orf_df = orf_df.drop(['Metadata_Symbol_y', 'Metadata_Symbol'], axis=1)

In [51]:
orf_df.shape

(82483, 256)

In [52]:
#Renaming the ['Metadata_Symbol_y'] and ['Metadata_NCBI_Gene_ID_y'] to be used in the further steps 
orf_df = orf_df.rename(columns={'Metadata_NCBI_Gene_ID_x':'Metadata_NCBI_Gene_ID', 'Metadata_Symbol_x':'Metadata_Symbol'})

In [55]:
orf_df.shape

(82483, 256)

In [56]:
orf_df.to_parquet('C:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\CRISPRPipelineValidation\\profiles_ORF.parquet')

#### Add annotations - I did not run this since the profiles that I have already has the annotations in it

In [4]:
#orf_metdata_df = pd.read_csv(
    #"../00.download-and-process-annotations/output/orf_metadata.tsv.gz", sep="\t"
#)
#compound_metadata_df = pd.read_csv(
    #"../datasets/metadata/compound.csv.gz", usecols=["Metadata_JCP2022"]
#).#assign(
    #Metadata_pert_type=lambda x: np.where(
     #   x["Metadata_JCP2022"] == "JCP2022_999999", "empty", "poscon"
 #   )
#)

#metadata_df = pd.concat(
 #   [
        #orf_metdata_df,
        #compound_metadata_df
  #  ],
    #join="outer",
    #ignore_index=True,
#)

#orf_df = orf_df.merge(metadata_df, on="Metadata_JCP2022", how="inner")
#orf_df.shape

(81660, 750)

#### Remove empty wells

In [57]:
orf_df = utils.remove_empty_wells(orf_df)
orf_df.shape

(81493, 256)

#### Remove `poscon` wells.

In [58]:
orf_df = orf_df.query('Metadata_pert_type!="poscon"').reset_index(drop=True)
orf_df.shape

(79563, 256)

#### Remove `BAD CONSTRUCT` profiles

In [59]:
orf_df = orf_df.query('Metadata_broad_sample!="BAD CONSTRUCT"').reset_index(drop=True)
orf_df.shape

(79563, 256)

#### Remove featues with `nan` values.
These need to be removed as the `nan` values will cause the mean average precision calculation to fail.

In [60]:
orf_df = utils.remove_nan_features(orf_df)

Removed nan features: []


#### Remove low infection effiency wells

In [61]:
# Add platemap name

platemap_df = pd.read_csv(
    "../00.download-and-process-annotations/input/experiment-metadata.tsv",
    sep="\t",
    usecols=["Plate_Map_Name", "Assay_Plate_Barcode"],
).rename(
    columns={
        "Plate_Map_Name": "Metadata_plate_map_name",
        "Assay_Plate_Barcode": "Metadata_Plate",
    }
)

orf_df = orf_df.merge(platemap_df, on="Metadata_Plate", how="left")

orf_df = utils.remove_low_infection_efficiency_wells(orf_df)
orf_df.shape

(72345, 257)

### Calculate mAP for each ORF perturbation

In [62]:
# Adding a new column for negative control
orf_df["Metadata_negcon"] = np.where(orf_df["Metadata_pert_type"] == "negcon", 1, 0)

In [63]:
pos_sameby = ["Metadata_JCP2022"]
pos_diffby = []
neg_sameby = ["Metadata_Plate"]
neg_diffby = ["Metadata_negcon"]

In [64]:
metadata_df = utils.get_metadata(orf_df)
feature_df = utils.get_featuredata(orf_df)
feature_values = feature_df.values

In [65]:
result = average_precision(
    metadata_df, feature_values, pos_sameby, pos_diffby, neg_sameby, neg_diffby, batch_size=batch_size
)

INFO:2024-08-22 17:36:48,243:copairs:Indexing metadata...
INFO:2024-08-22 17:36:48,352:copairs:Finding positive pairs...
INFO:2024-08-22 17:36:49,295:copairs:Finding negative pairs...
INFO:2024-08-22 17:36:50,336:copairs:Computing positive similarities...
INFO:2024-08-22 17:36:52,407:copairs:Computing negative similarities...
INFO:2024-08-22 17:36:53,636:copairs:Building rank lists...
INFO:2024-08-22 17:36:55,610:copairs:Computing average precision...
INFO:2024-08-22 17:36:55,721:copairs:Creating result DataFrame...
INFO:2024-08-22 17:36:55,734:copairs:Finished.


In [66]:
# Remove negcon
result = result.query('Metadata_pert_type!="negcon"').reset_index(drop=True)

In [67]:
agg_result = (
    mean_average_precision(result, pos_sameby, null_size=null_size, threshold=fdr, seed=12527)
    .rename(columns={'average_precision': 'mean_average_precision'})
)

INFO:2024-08-22 17:37:18,859:copairs:Computing null_dist...
INFO:2024-08-22 17:37:18,928:copairs:Computing p-values...


In [68]:
agg_result.to_csv(f"C:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\CRISPRPipelineValidation\\PhenotypicActivity_ORF.csv.gz", index=False)